<a href="https://colab.research.google.com/github/kmalik22/colabs/blob/main/transformer_training_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
import torch
import numpy as np
import torch.nn.functional as F
import itertools
from typing import Optional
from torch.utils.data import IterableDataset, DataLoader
torch.set_printoptions(linewidth=250, precision=3)

# Modules

In [253]:
class MLP(torch.nn.Module):
  def __init__(self, d_model, d_ffn):
    super().__init__()
    self.d_model = d_model
    self.d_ffn = d_ffn
    self.fc1 = torch.nn.Linear(in_features=d_model, out_features=d_ffn, bias=False)
    torch.nn.init.trunc_normal_(self.fc1.weight, mean=0, std=1/d_model**0.5)
    self.fc2 = torch.nn.Linear(in_features=d_ffn, out_features=d_model, bias=False)
    torch.nn.init.trunc_normal_(self.fc2.weight, mean=0, std=1/d_ffn**0.5, a=-3, b=3)

  def forward(self, x: torch.Tensor):
    fc1_out = self.fc1(x)
    relu_out = torch.nn.functional.relu(fc1_out)
    fc2_out = self.fc2(relu_out)
    return fc2_out

class MultiHeadAttention(torch.nn.Module):
  def __init__(self, d_model, num_heads):
    super().__init__()
    self.d_model = d_model
    self.num_heads = num_heads
    self.head_dim = d_model//num_heads
    self.wqkv = torch.nn.Linear(d_model, int(3*d_model), bias=False)
    torch.nn.init.trunc_normal_(self.wqkv.weight, mean=0, std=1/d_model**0.5, a=-3, b=3)
    self.wo = torch.nn.Linear(d_model, d_model)
    torch.nn.init.trunc_normal_(self.wo.weight, mean=0, std=1/d_model**0.5, a=-3, b=3)
    # TODO: add rope

  def scaled_dot_product_attn(self, q, k, v):
    # q, k, v = (bsz, num_heads, seqlen, head_dim)
    # qk.t()/sqrt(head_dim)
    # causal mask
    # softmax
    # @ v
    seqlen = q.shape[2]
    attn_wts = q @ k.transpose(2,3)  #(bsz,num_h,seqlen,head_dim) (bsz,num_h,head_dim,seqlen) -> (bsz,num_h,seqlen,seqlen)
    # create mask, do torch.where. 1=use attn wts, else use -inf
    mask = torch.tril(torch.ones(seqlen,seqlen)).to(torch.bool)
    masked_attn_wts = torch.where(mask, attn_wts, float('-inf')) # (bsz,num_heads,seqlen,seqlen)

    # softmax TODO check once
    softmax_wts = torch.nn.functional.softmax(masked_attn_wts,dim=-1)
    # (bsz,num_heads,seqlen,seqlen) @ (bsz,num_heasds,seqlen,head_dim) -> (bsz,num_heads,seqlen,head_dim)
    return softmax_wts @ v


  def forward(self, x:torch.Tensor):
    #x = (bsz, seqlen, d_model)
    bsz, seqlen, d_model = x.shape
    assert d_model == self.d_model
    wqkv_out = self.wqkv(x) # (bsz, seqlen, 3*d_model)
    # rearrange so seqlen,head_dim are the last two dims
    wqkv_out = wqkv_out.reshape(bsz, 3, self.num_heads, seqlen, self.head_dim)
    q = wqkv_out[:,0,:,:,:] # (bsz, num_heads, seqlen, head_dim)
    k = wqkv_out[:,1,:,:,:] # (bsz, num_heads, seqlen, head_dim)
    v = wqkv_out[:,2,:,:,:] # (bsz, num_heads, seqlen, head_dim)
    # TODO: add rope to q and k
    self_attn_out = self.scaled_dot_product_attn(q, k, v) #(bsz, num_heads, seqlen, head_dim)
    # transpose
    self_attn_out = self_attn_out.transpose(1,2) # (bsz,seqlen,num_heads,head_dim)
    # concat the last dim
    self_attn_out = self_attn_out.reshape(bsz,seqlen,-1)
    # wo
    self_attn_out = self_attn_out.reshape(bsz, seqlen, d_model)
    wo_out = self.wo(self_attn_out)
    return wo_out


class TransformerBlock(torch.nn.Module):
  def __init__(self, d_model, d_ffn, num_heads, max_seqlen):
    super().__init__()
    self.mlp_norm = torch.nn.RMSNorm(normalized_shape=d_model)
    self.mlp = MLP(d_model=d_model, d_ffn=d_ffn)
    self.attn_norm = torch.nn.RMSNorm(normalized_shape=d_model)
    self.attn = MultiHeadAttention(d_model=d_model,num_heads=num_heads)


  def forward(self, x:torch.Tensor):
    # norm --> attn, add back to original --> norm --> mlp, add back
    attn_norm_out = self.attn_norm(x)
    print(f"attn_norm_out: {attn_norm_out[0,0,:]}")
    attn_out = self.attn(attn_norm_out)
    print(f"attn_out: {attn_out[0,0,:]}")
    attn_out_post_resid = x + attn_out
    mlp_norm_out = self.mlp_norm(attn_out_post_resid)
    mlp_out = self.mlp(mlp_norm_out)
    retval = mlp_out + attn_out_post_resid
    return

class Transformer(torch.nn.Module):
  def __init__(self, d_model, d_ffn, vocab_size, num_layers, max_seqlen, num_heads, simple_pos_embed=False):
    super().__init__()
    self.max_seqlen = max_seqlen
    self.embedding = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=d_model)
    torch.nn.init.trunc_normal_(self.embedding.weight, mean=0, std=1/d_model**0.5, a=-3, b=-3)

    self.simple_pos_embed = simple_pos_embed
    if simple_pos_embed:
      self.pos_embedding = torch.nn.Embedding(num_embeddings=max_seqlen, embedding_dim=d_model)

    self.output_layer = torch.nn.Linear(in_features=d_model, out_features=vocab_size)
    torch.nn.init.trunc_normal_(self.output_layer.weight, mean=0, std=1/d_model**0.5, a=-3, b=3)
    self.layers = torch.nn.ModuleList(
        [TransformerBlock(d_model=d_model, d_ffn=d_ffn, num_heads=num_heads, max_seqlen=max_seqlen) for _ in range(num_layers)]
    )
    # TODO: do proper init
    # add attention
    # add layer norms

  def forward(self, tokens: torch.Tensor):
    # tokens: (bsz,seqlen,d_model)
    seqlen = tokens.shape[1]
    curr_out = self.embedding(tokens)
    if self.simple_pos_embed:
      curr_out = curr_out + self.pos_embedding(torch.arange(seqlen))
    for l in self.layers:
      curr_out = curr_out + l(curr_out)
    logits = self.output_layer(curr_out)  # bsz,seqlen,vocab_size
    return logits


# Test code

In [254]:
tblock_test = TransformerBlock(d_model=D_MODEL, d_ffn=D_FFN, num_heads=NUM_HEADS, max_seqlen=SEQLEN)
input_act = torch.randn(BSZ, SEQLEN, D_MODEL)
tblock_out = tblock_test(input_act)
assert tblock_out.shape == (BSZ, SEQLEN, D_MODEL)

input_act_trunc = input_act[:,:2,:]
tblock_out2 = tblock_test(input_act_trunc)

print(input_act[0,0,:])
print(input_act_trunc[0,0,:])

print(tblock_out[0,0,:])
print(tblock_out2[0,0,:])

attn_norm_out: tensor([ 0.452,  0.205, -1.575, -1.386, -1.095, -0.231,  1.232, -1.143,  0.256,  0.728, -0.624,  0.743,  1.186,  1.459,  1.153,  0.937], grad_fn=<SliceBackward0>)
attn_out: tensor([ 0.075,  0.088, -0.319,  0.960,  1.338,  0.489, -0.344,  0.323,  1.019,  0.494,  0.272, -0.042,  0.438, -0.529,  0.839, -0.159], grad_fn=<SliceBackward0>)


AttributeError: 'NoneType' object has no attribute 'shape'

# Dataloader

In [229]:
EOS_TOKEN=-1

class DigitDataset(IterableDataset):
  def __init__(self, num_digits=10):
    self.num_digits = 10

  def __iter__(self):
    while True:
      for i in range(self.num_digits):
        yield i


def collate_batch_digits(batch, seqlen, bsz):
  x = torch.tensor(batch, dtype=torch.long).view(bsz, seqlen)
  y = torch.roll(x, shifts=-1, dims=(1,))
  y[:,-1] = EOS_TOKEN
  return x,y

# dataset that spits out 8 unsorted digits, then sorts them all

class SortedDigitsDataset(IterableDataset):
  # outputs a full batch
  def __init__(self, num_digits=10, max_seqlen=8, eos_token=EOS_TOKEN):
    self.num_digits = 10
    self.max_seqlen = max_seqlen
    self.eos_token = eos_token

  def __iter__(self):
    while True:
      half = int(self.max_seqlen//2)
      half_batch = torch.randint(low=0,high=(self.num_digits-1), size=(half,))
      sorted = torch.sort(half_batch).values
      batch = torch.cat([half_batch, sorted])
      targets = torch.roll(batch, shifts=-1)
      targets[:int(half)-1] = self.eos_token
      targets[-1] = self.eos_token
      #print(f"batch:{batch}\n targets:{targets}\n")
      yield (batch, targets)


# Dataloader test

In [230]:
sorted_digits_dataset = SortedDigitsDataset(max_seqlen=SEQLEN)
sorted_dataloader = DataLoader(dataset=sorted_digits_dataset, batch_size=BSZ, shuffle=False)

an_iter = iter(sorted_digits_dataset)
while True:
  print(next(an_iter))
  break

(tensor([4, 8, 4, 4, 3, 1, 8, 8, 1, 3, 4, 4, 4, 8, 8, 8]), tensor([-1, -1, -1, -1, -1, -1, -1,  1,  3,  4,  4,  4,  8,  8,  8, -1]))


# Constants

In [231]:
D_MODEL=16
D_FFN=64
NUM_LAYERS=2
VOCAB_SIZE=16
SEQLEN=16
BSZ=32
NUM_HEADS=4
HEAD_DIM=D_MODEL//NUM_HEADS

# Dataloader

In [232]:
sorted_digits_dataset = SortedDigitsDataset(max_seqlen=SEQLEN)
sorted_dataloader = DataLoader(dataset=sorted_digits_dataset, batch_size=BSZ, shuffle=False)
dl_iter = iter(sorted_dataloader)

# Trainer

In [233]:

STEPS=5001

model = Transformer(d_model=D_MODEL, d_ffn=D_FFN, vocab_size=VOCAB_SIZE, num_layers=NUM_LAYERS, num_heads=NUM_HEADS, max_seqlen=SEQLEN, simple_pos_embed=True)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=0.001, weight_decay=0)

#digits = DigitDataset()
#digit_dataloader = DataLoader(dataset=digits, batch_size=SEQLEN*BSZ, shuffle=False, collate_fn=lambda b:collate_batch_digits(batch=b, seqlen=SEQLEN, bsz=BSZ))
#dl_iter = iter(digit_dataloader)


for step in range(STEPS):
  optimizer.zero_grad()
  tokens, targets = next(dl_iter) #targets = (bsz, seqlen)   tokens=(bsz, seqlen)
  logits = model(tokens)  # (bsz, seqlen, vocab_size)
  vocab_size = logits.shape[-1]
  loss = torch.nn.functional.cross_entropy(input=logits.view(-1, vocab_size), target=targets.view(-1), ignore_index=EOS_TOKEN)
  if step %100 == 0:
    print(f"{step=} {loss=}")
    #print(f"tokens:{tokens[0,:]} \n targets:{targets[0,:]} \n\n")
    #print(f"tokens:{tokens[0,:]} \n targets:{targets[0,:]} \n probs:\n{logits[0,:,:]}\n")
  loss.backward()
  optimizer.step()




/usr/local/lib/python3.12/dist-packages/torch/nn/init.py:295: UserWarning: mean is more than 2 std from [a, b] in nn.init.trunc_normal_. The distribution of values may be incorrect.
  return _no_grad_trunc_normal_(tensor, mean, std, a, b, generator=generator)


step=0 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=100 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=200 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=300 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=400 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=500 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=600 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=700 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=800 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=900 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=1000 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=1100 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=1200 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=1300 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=1400 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=1500 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=1600 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=1700 loss=tensor(nan, grad_fn=<NllLossBackward0>)
step=1800 loss=tensor(

In [171]:
# generation
# dumb way: 1 token at a time. get logits, do softmax, sample token. feed in full generated thing back

# kv cache way: attention takes in a kv cache which are pre-computed k and v values for previous tokens



# Simple generation

In [219]:
def softmax_with_temp_last_dim(logits: torch.Tensor, temp: float=1.0, eps=1e-6):
  # softmax across the innermost (last) dimension
  # compute max
  # subtract max
  # numerator = exponentiate with temperature
  # denominator = sum of numerator
  #import pdb; pdb.set_trace()
  # logits: (..., vocab_size)
  maxval = torch.amax(logits, dim=-1).unsqueeze(-1)
  max_subtracted_logits = logits - maxval
  scaled_max_subtracted_logits = max_subtracted_logits/torch.Tensor([temp+eps])
  numerator = torch.exp(scaled_max_subtracted_logits)
  denominator = torch.sum(numerator, dim=-1).unsqueeze(-1)
  return numerator/(denominator + eps)


def retain_top_p(probs: torch.Tensor, p: float):
  # retain only probabilities with mass = top_p
  assert len(probs.shape) == 1 # batch later
  sorted_probs = torch.sort(probs, descending=True)
  sorted_probs_values = sorted_probs.values
  sorted_probs_indices = sorted_probs.indices
  cumsum_probs = list(torch.cumsum(sorted_probs_values, dim=0))
  for i, prob in enumerate(cumsum_probs):  # super ugly, need vectorized version
    if prob > p:
      break
  not_useful_indices = sorted_probs_indices[(i+1):]
  if len(not_useful_indices) > 0:
    probs[not_useful_indices] = 0 # TODO check here
  return probs



def sample(logits: torch.Tensor, top_p: Optional[float]=None, temp: float = 1.0):
  # logits: (vocab_size,)
  probs = softmax_with_temp_last_dim(logits=logits, temp=temp)
  print(probs)
  if top_p:
    probs = retain_top_p(probs=probs, p=top_p)
  sampled = np.random.multinomial(n=1, pvals=probs)
  return int(np.argwhere(sampled)[0][0])


def generate(the_model: torch.nn.Module, input_tokens: torch.Tensor, toks_to_generate: int, temp=1.0):
  # for each token, run forward
    # run forward get logits
    # convert logits to probabilities
    # sample from probabilities
    # update input tokens
  generated_toks = []
  for _ in range(toks_to_generate):
    logits = the_model(input_tokens)
    #print(f"logits.shape:{logits.shape}")
    print(f"full_logits:{logits}")
    logits = logits.squeeze(0)[-1,:] #unsqueeze to remove batch dim. -1 for last token
    print(f"logits:{logits}")
    sampled_token = sample(logits, temp=temp)
    generated_toks.append(sampled_token)
    #print(input_tokens.shape)
    #print(torch.tensor([sampled_token]).unsqueeze(0).shape)
    input_tokens = torch.cat([input_tokens.squeeze(0), torch.tensor([sampled_token])]).unsqueeze(0)
  return generated_toks


# Simple tests

In [224]:
input, targets = next(dl_iter) #get the first entry only
input = input[:1,:]
targets = targets[:1,:]
print(input)
print(targets)
trunc_input = input[:1,:int(SEQLEN//2)]
print(trunc_input)

tensor([[8, 7, 5, 4, 7, 4, 4, 1, 1, 4, 4, 4, 5, 7, 7, 8]])
tensor([[-1, -1, -1, -1, -1, -1, -1,  1,  4,  4,  4,  5,  7,  7,  8, -1]])
tensor([[8, 7, 5, 4, 7, 4, 4, 1]])


In [225]:
logits_raw = model(input)
print(f"raw_logits:{logits_raw}")
print(f"logits for last:{logits_raw.squeeze(0)[7,:]}")

raw_logits:tensor([[[-8.717e+00,  3.113e+00, -3.977e+01, -9.374e+00,  2.456e+01,  3.480e+01,  8.431e+00,  3.421e+00,  4.940e+01, -3.145e+01, -2.733e+01, -1.041e+01, -1.330e+01, -4.251e+00, -5.668e+00, -1.252e+01],
         [ 1.035e+01,  8.766e-01, -3.859e+01, -1.126e+01,  3.591e+01,  3.384e+01, -6.838e+00, -1.821e+01,  3.177e+01, -2.001e+01, -1.713e+01, -9.427e+00, -1.330e+01, -4.967e+00, -6.192e+00, -2.626e+00],
         [-1.887e+01,  1.556e+01,  5.055e-01, -1.758e+01, -2.055e+01,  7.803e+00,  2.347e+01,  1.953e+01,  2.444e+00, -2.635e+01, -1.841e+01,  1.042e+00,  3.919e-01, -1.387e+01, -5.886e+00,  8.695e-01],
         [-4.570e+00,  1.598e+01, -7.507e+00,  9.737e+00,  1.143e+01,  8.296e+00, -1.097e+01, -5.354e+00,  9.391e+00, -2.693e+01, -1.223e+01, -8.187e+00, -7.584e-01, -8.673e+00, -1.328e+01, -2.021e+01],
         [ 4.740e+00,  2.614e+01, -2.655e+01,  7.097e+00, -1.026e+01, -8.144e+00,  1.667e+01,  2.233e+01,  1.499e+01, -2.901e+01, -1.381e+01, -1.920e+01, -3.452e+01, -1.587e+01,

In [226]:
# test for sampling functions
#logits = torch.tensor([5,1,1,5,1])
#sample(logits, temp=0)

with torch.no_grad():
  # test for full generation
  #input = next(dl_iter)[0][0,:].unsqueeze(0) #get the first entry only
  generated = generate(the_model=model, input_tokens=trunc_input, toks_to_generate=1, temp=1)
  print(generated)


full_logits:tensor([[[ -0.638,  19.998, -19.026,   3.030,  -0.644,   4.044,   4.200,  18.733,  26.821, -27.488, -19.970,  -7.010, -19.914,  -5.486, -11.259, -21.677],
         [ 16.976,   4.150, -41.736,  -7.956,  35.129,  31.589,  -8.484, -10.738,  33.263, -25.014, -20.447,  -9.452, -18.661,  -6.777, -10.125, -10.792],
         [ 30.958,   6.546, -40.985, -13.567,  33.140,  21.235,  -8.875,  -2.774,  31.594, -10.831, -22.016, -11.067, -18.855,   2.142,  -5.156,  -9.298],
         [ -3.273,   9.484,  -7.698,  -5.889,  27.474,  28.073,  -9.269, -21.469,   5.784, -22.441, -17.575, -11.469,  11.179,  -5.281,  -3.860,  -7.326],
         [ 10.631,  -0.351, -38.579,  -6.840,  34.729,  25.680, -12.670, -10.089,  36.073,  -7.165, -19.532, -10.590, -17.231,  -2.687,  -2.040,  -4.257],
         [ -4.842,  11.866,  -7.065,  -0.214,  -3.651,   8.947,  12.381,  -4.340,  -5.157, -19.088,  -4.776, -14.768,  -8.145, -12.156, -12.376, -17.710],
         [ 26.398,  -2.742, -41.691,  -4.501,  52.266,  24

# Sample

In [62]:
probs

tensor([0.487, 0.009, 0.009, 0.487, 0.009])

In [57]:
int(np.argwhere(np.random.multinomial(n=1, pvals=probs))[0][0])

3

In [133]:
model

Transformer(
  (embedding): Embedding(16, 16)
  (pos_embedding): Embedding(16, 16)
  (output_layer): Linear(in_features=16, out_features=16, bias=True)
  (layers): ModuleList(
    (0-1): 2 x TransformerBlock(
      (mlp_norm): RMSNorm((16,), eps=None, elementwise_affine=True)
      (mlp): MLP(
        (fc1): Linear(in_features=16, out_features=64, bias=False)
        (fc2): Linear(in_features=64, out_features=16, bias=False)
      )
      (attn_norm): RMSNorm((16,), eps=None, elementwise_affine=True)
      (attn): MultiHeadAttention(
        (wqkv): Linear(in_features=16, out_features=48, bias=False)
        (wo): Linear(in_features=16, out_features=16, bias=True)
      )
    )
  )
)